<a href="https://colab.research.google.com/github/monsieurshiva/AdvancedDeepLearning/blob/main/CustomLayer/Building_Custom_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Custom Dense Layer

We will create a custom layer that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) class.

## Imports

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Layer


## Custom Layer with weights

To make custom layer that is trainable, we need to define a class that inherits the [Layer](https://keras.io/api/layers/base_layer/#layer-class) base class from Keras. The Python syntax is shown below in the class declaration. This class requires three functions: `__init__()`, `build()` and `call()`. These ensure that our custom layer has a *state* and *computation* that can be accessed during training or inference.

In [ ]:
class SimpleDense(Layer):

    def __init__(self, units=32, activation=None):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units
        # define the activation to get from the built-in activation layers in Keras
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        # initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return self.activation(tf.matmul(inputs, self.w) + self.b)

Now we can use our custom layer like below:

In [ ]:
# declare an instance of the class
my_dense = SimpleDense(units=1)

# define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.03951098]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Let's then try using it in simple network:

In [ ]:
# define the dataset
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)


# use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1,activation='relu')
model = tf.keras.Sequential([my_layer])

# configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(tf.expand_dims(xs, axis=-1), ys, epochs=500,verbose=0)

# perform inference
print(model.predict([10.0]))

# see the updated state of the variables
print(my_layer.variables)

1/1 [==============================] - 0s 316ms/step
[[17.953257]]
[<tf.Variable 'simple_dense_2/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.8517336]], dtype=float32)>, <tf.Variable 'simple_dense_2/bias:0' shape=(1,) dtype=float32, numpy=array([-0.564079], dtype=float32)>]


## Implementing a Simple Quadratic Layer

It has 3 state variables: $a$, $b$ and $c$. The computation returned is $ax^2 + bx + c$.

In [ ]:
class SimpleQuadratic(Layer):

    def __init__(self, units=32, activation=None):
        '''Initializes the class and sets up the internal variables'''
        super(SimpleQuadratic, self).__init__()
        self.units = units
        # define the activation to get from the built-in activation layers in Keras
        self.activation = tf.keras.activations.get(activation)
    
    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # a and b should be initialized with random normal, c (or the bias) with zeros.
        # remember to set these as trainable.
        a_init = tf.random_normal_initializer()
        self.a = tf.Variable(name="kernel",
            initial_value=a_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)
        
        b_init = tf.random_normal_initializer()
        self.b = tf.Variable(name="kernel",
            initial_value=b_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)
        # initialize the biases
        c_init = tf.zeros_initializer()
        self.c = tf.Variable(name="bias",
            initial_value=c_init(shape=(self.units,), dtype='float32'),
            trainable=True)
   
    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        # Remember to use self.activation() to get the final output
        return self.activation(tf.matmul(tf.math.square(inputs),self.a)+tf.matmul(inputs,self.b)+self.c)

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  SimpleQuadratic(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

11490434/11490434 [==============================] - 3s 0us/step
Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2671 - accuracy: 0.9212
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1297 - accuracy: 0.9618
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1000 - accuracy: 0.9689
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0821 - accuracy: 0.9745
Epoch 5/5
313/313 [==============================] - 3s 3ms/step - loss: 0.0788 - accuracy: 0.9764


[0.0788261890411377, 0.9764000177383423]